# Music Recommendation System

Music recommender system can suggest songs to users based on their listening pattern.

Datasetlinks https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data

# Importing Essential Libraries

I have created **Reommenders** library from where I will be importing few methods. <br>
If you want to have a look please check out in the **03- Music Recommendation System** Folder.

In [17]:
import Recommenders as Recommenders

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time

# Loading all Music Data

**What does parsing date mean in Python?** <br>
Its parse function can figure out what format a string is in without having to specify the format like you do with datetime

In [18]:
members = pd.read_csv('members.csv',parse_dates=["registration_init_time","expiration_date"])
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,2011-08-20,2017-09-20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,2015-06-28,2017-06-22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,2016-04-11,2017-07-12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,2015-09-06,2015-09-07
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,2017-01-26,2017-06-13


In the above table: <br>
**msno**:userid <br>
**bd**: age. Note: this column has outlier values, please use your judgement<br>
**registered_via**: registration method

In [21]:
df_train = pd.read_csv('train.csv')
print(df_train.shape)
df_train.head()

(7377418, 6)


,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


**target**: this is the target variable. target=1 means there are recurring listening event(s) triggered within a month after the user’s very first observable listening event, target=0 otherwise .

In [23]:
df_songs = pd.read_csv("songs.csv")
print(df_songs.shape)
df_songs.head()

(2296320, 7)


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [14]:
df_songs_extra = pd.read_csv("song_extra_info.csv")
df_songs_extra.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


**isrc** - International Standard Recording Code, theoretically can be used as an identity of a song. However, what worth to note is, ISRCs generated from providers have not been officially verified; therefore the information in ISRC, such as country code and reference year, can be misleading/incorrect. Multiple songs could share one ISRC since a single recording could be re-published several times.

# Creating our own Data

In [24]:
res = df_train.merge(df_songs[['song_id','song_length','genre_ids','artist_name','language']], on=['song_id'], how='left')
print(res.shape)
res.head()

(7377418, 10)


,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,language
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,52.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,52.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,52.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,-1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,52.0


In [25]:
train = res.merge(df_songs_extra,on=['song_id'],how = 'left')
print(train.shape)
train.head()

(7377418, 12)


,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,language,name,isrc
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,52.0,Good Grief,GBUM71602854
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,52.0,Lords of Cardboard,US3C69910183
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,52.0,Hip Hop Is Dead(Album Version (Edited)),USUM70618761
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,-1.0,Disco Africa,GBUQH1000063
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,52.0,Sleep Without You,QM3E21606003


We also want one column where it shows the number of times user has listen to that particular song. <br>
So we will make a dataframe ("song1") where we will count the number of time the song has been listened. <br>
Then we will add this column ("listen_count") to our origninal dataset with the primary key as "name". <br>

In [34]:
song_id = train.loc[:,["name","target"]]
song1 = song_id.groupby(["name"],as_index=False).count().rename(columns = {"target":"listen_count"})
print(song1.shape)
song1.head()

(234144, 2)


,name,listen_count
0,24karats –type S-(與放浪兄弟EXILE、DOBELMAN合作超跳嘻哈單曲),5
1,A Gentle Dissolve (Shawn Lee Ping Pong Orches...,1
2,A-CHA,2
3,Adicto,1
4,Be Your Sound (Radio Edit),10


In [35]:
dataset=train.merge(song1,on=['name'],how= 'left')
dataset.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,language,name,isrc,listen_count
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,52.0,Good Grief,GBUM71602854,293.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,52.0,Lords of Cardboard,US3C69910183,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,52.0,Hip Hop Is Dead(Album Version (Edited)),USUM70618761,4.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,-1.0,Disco Africa,GBUQH1000063,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,52.0,Sleep Without You,QM3E21606003,412.0


In [30]:
df=pd.DataFrame(dataset)
df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,language,name,isrc,listen_count
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,52.0,Good Grief,GBUM71602854,293.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,52.0,Lords of Cardboard,US3C69910183,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,52.0,Hip Hop Is Dead(Album Version (Edited)),USUM70618761,4.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,-1.0,Disco Africa,GBUQH1000063,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,52.0,Sleep Without You,QM3E21606003,412.0


Let's now keep only usefull columns and drop others.

In [31]:
df.drop(columns=['source_system_tab','source_screen_name','source_type','target','isrc'],axis=1,inplace=True)
df=df.rename(columns={'msno':'user_id'})

In [32]:
df.head()

,user_id,song_id,song_length,genre_ids,artist_name,language,name,listen_count
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,206471.0,359,Bastille,52.0,Good Grief,293.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,284584.0,1259,Various Artists,52.0,Lords of Cardboard,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,225396.0,1259,Nas,52.0,Hip Hop Is Dead(Album Version (Edited)),4.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,255512.0,1019,Soundway,-1.0,Disco Africa,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,187802.0,1011,Brett Young,52.0,Sleep Without You,412.0


#  Data Preprocessing

In [37]:
df.shape

(7377418, 8)

In [38]:
df.isnull().sum()

user_id              0
song_id              0
song_length        114
genre_ids       118455
artist_name        114
language           150
name              1457
listen_count      1457
dtype: int64

In [50]:
#filling null values
df['song_length'].fillna('0',inplace=True)
df['genre_ids'].fillna('0',inplace=True)
df['artist_name'].fillna('none',inplace=True)
df['language'].fillna('0',inplace=True)
df['name'].fillna('none',inplace=True)
df['listen_count'].fillna('0',inplace=True)

In [51]:
df.isnull().sum()

user_id         0
song_id         0
song_length     0
genre_ids       0
artist_name     0
language        0
name            0
listen_count    0
dtype: int64

In [52]:
print("Total no of songs:",len(df))

Total no of songs: 7377418


# Create a subset of the dataset 
On this we will perform our anaysis instead of doing on whole datasets.

In [54]:
df = df.head(10000)

#Merge song title and artist_name columns to make a new column ("song")
df['song'] = df['name'].map(str) + " - " + df['artist_name']

<ipython-input-54-9ee857c03aa1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['song'] = df['name'].map(str) + " - " + df['artist_name']


In [56]:
print(df.shape)
df.head()

(10000, 9)


,user_id,song_id,song_length,genre_ids,artist_name,language,name,listen_count,song
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,206471.0,359,Bastille,52.0,Good Grief,293.0,Good Grief - Bastille
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,284584.0,1259,Various Artists,52.0,Lords of Cardboard,1.0,Lords of Cardboard - Various Artists
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,225396.0,1259,Nas,52.0,Hip Hop Is Dead(Album Version (Edited)),4.0,Hip Hop Is Dead(Album Version (Edited)) - Nas
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,255512.0,1019,Soundway,-1.0,Disco Africa,1.0,Disco Africa - Soundway
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,187802.0,1011,Brett Young,52.0,Sleep Without You,412.0,Sleep Without You - Brett Young


# Showing the most popular songs in the dataset
The column listen_count denotes the no of times the song has been listened.Using this column, we’ll find the dataframe consisting of popular songs:

In [57]:
song_gr = df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()

In [58]:
song_gr['listen_count'].sum()

10000

In [63]:
song_gr = df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_gr['listen_count'].sum()
song_gr['percentage']  = song_gr['listen_count'].div(grouped_sum)*100
song_gr.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
3166,告白氣球 - 周杰倫 (Jay Chou),62,0.62
3630,帥到分手 - 周湯豪 (NICKTHEREAL),54,0.54
5194,謝謝妳愛我 (Thanks For Your Love) - 謝和弦 (R-chord),53,0.53
428,Closer - The Chainsmokers,38,0.38
2866,你，好不好？ (How Have You Been?) - Eric 周興哲,38,0.38
...,...,...,...
5581,착한여자(GOOD TO YOU) - 2NE1,1,0.01
5583,촌스럽게 왜 이래 - K.Will,1,0.01
5584,칼로리 송 (통통한거라 믿고 싶겠지) Calorie Song - Coffee Boy,1,0.01
5585,헤어질걸 알기에(i know| we will breake up ) - Dia Tr...,1,0.01


# Count number of unique users in the dataset

In [64]:
users = df['user_id'].unique()
print("The no. of unique users:", len(users))

The no. of unique users: 1622


# Count the number of unique songs in the dataset

In [73]:
songs = df['song'].unique()
print("The no. of unique songs:", len(songs))

The no. of unique songs: 5587


# Create a song recommender

In [74]:
train_data, test_data= train_test_split(df, test_size = 0.20, random_state = 0)

In [79]:
print(train_data.shape)
train_data.head()

(8000, 9)


,user_id,song_id,song_length,genre_ids,artist_name,language,name,listen_count,song
7389,6zx0HeEdbSL3AGTjtgHStAyC91v8XejM5EdY7spnScA=,N7GrR6FPc1gvScpFr702q/hqD67xziT4944fNmQPy/c=,292989.0,1609,Club cheval,52.0,Discipline,12.0,Discipline - Club cheval
9275,L2kSbspTMs3NODBwFJhSRwJ+y9KFJ+BrOrdjvvtplvY=,a4TbK5V15pj3YZUOGa9h2U3t0OsE+3aiFw41mNlcgcw=,248790.0,465,吳汶芳 (Fang Wu),3.0,孤獨的總和 (Accumulated Loneliness),2753.0,孤獨的總和 (Accumulated Loneliness) - 吳汶芳 (Fang Wu)
2995,MfAu5rJVFdtHAuslq5EofZM3e4MwGA4A8OJ45FLHmKM=,MZhCRo/HtYw5BBh+adG7iImUFvTNQupj9iq/CPmPfhE=,255895.0,465,劉若英 (Rene Liu),3.0,不能跟情人說的話,449.0,不能跟情人說的話 - 劉若英 (Rene Liu)
5316,QZKu9OYJqa1J2azCzw6amqHc0SgBZSLiUN4FLKXJxgA=,dFlAZd5wSJLMyWJbBJSKmzq/32xpj89CSWy5iavjIGQ=,210860.0,465,葉秉桓,3.0,想太多 (Over Thinking),27.0,想太多 (Over Thinking) - 葉秉桓
356,wRKpx+fzd5Eb0c73MIooZzvPpdOyOnMg1hAKh5dohaw=,IlNTgOUmmXElav8hTkv9ps73MvfeAgAGEQMGDEYD5xs=,270001.0,458,林宥嘉 (Yoga Lin),3.0,寵兒,973.0,寵兒 - 林宥嘉 (Yoga Lin)


In [80]:
print(test_data.shape)
test_data.head()

(2000, 9)


,user_id,song_id,song_length,genre_ids,artist_name,language,name,listen_count,song
9394,xu9r1adsxG6vopl68JyNFkq7WesG2VkTdTkoM0BBqVc=,QZBm8SOwnEjNfCpgsKBBGPMGET6y6XaQgnJiirspW7I=,266448.0,458,楊丞琳 (Rainie Yang),3.0,年輪說,7769.0,年輪說 - 楊丞琳 (Rainie Yang)
898,07+a5Wl5ujG0rOjTrZ1nsd3VxpLuAsgEwN/pyXHwvRc=,PHqVEALMocWEArPeLdtc++lcSDdzXeXJoRIydT1s3dc=,208608.0,921,Jessie J,52.0,Flashlight,1553.0,Flashlight - Jessie J
2398,ZLYp5tEgfBz+3S+GHKf4oXsOyty9008S0w5M7wJOkNo=,Khe3Xi3r+D4GHaNnxb4ZL+TSGNF+4SkmxpnPG90tVY4=,170004.0,947,Isao Mori,-1.0,Satie Gymnopedie No.1 Lent Et Douloureux,14.0,Satie Gymnopedie No.1 Lent Et Douloureux - Isa...
5906,8NbvTe3FCtrI7uUrKMY+LPVee7w1TVePGXoiPiyOjOE=,FqssNUdbISQMw9O5IoLTVXg818vy9cN/8UpI3ESJqfY=,293128.0,465,Walk Off The Earth,52.0,Hello,6793.0,Hello - Walk Off The Earth
2343,67rPKrvEGak0BS4XRPmQSCSWDw85Vt6EFza1YeJOOco=,vBHRgZ/uZC8+KsrQ3W2pvVirD2ynPYR0baBs3cEZdNk=,201142.0,465,五月天 (Mayday),3.0,搖滾舞台,140.0,搖滾舞台 - 五月天 (Mayday)


# Creating Popularity based Music Recommendation
Using popularity_recommender class we made in Recommenders.py package, we create the list given below:

In [82]:
pm = Recommenders.popularity_recommender_py()                               #create an instance of the class
pm.create(train_data, 'user_id', 'song')

user_id1 = users[5]                                                          #Recommended songs list for a user
pm.recommend(user_id1)

,user_id,song,score,Rank
2659,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,告白氣球 - 周杰倫 (Jay Chou),53,1.0
3055,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,帥到分手 - 周湯豪 (NICKTHEREAL),47,2.0
4394,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,謝謝妳愛我 (Thanks For Your Love) - 謝和弦 (R-chord),47,3.0
2400,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,你，好不好？ (How Have You Been?) - Eric 周興哲,34,4.0
4678,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,默 - aMEI (張惠妹),29,5.0
351,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,Closer - The Chainsmokers,28,6.0
2883,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,好愛好散 (Blue Love Theme) - 陳勢安 (Andrew Tan),28,7.0
3107,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,後來的我們 (Here| After| Us) - 五月天 (Mayday),27,8.0
2874,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,好好 (想把你寫成一首歌) (Song About You) - 五月天 (Mayday),25,9.0
464,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,Don't Let Me Down - The Chainsmokers,23,10.0


In [85]:
user_id2 = users[45]
pm.recommend(user_id2)

,user_id,song,score,Rank
2659,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,告白氣球 - 周杰倫 (Jay Chou),53,1.0
3055,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,帥到分手 - 周湯豪 (NICKTHEREAL),47,2.0
4394,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,謝謝妳愛我 (Thanks For Your Love) - 謝和弦 (R-chord),47,3.0
2400,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,你，好不好？ (How Have You Been?) - Eric 周興哲,34,4.0
4678,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,默 - aMEI (張惠妹),29,5.0
351,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,Closer - The Chainsmokers,28,6.0
2883,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,好愛好散 (Blue Love Theme) - 陳勢安 (Andrew Tan),28,7.0
3107,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,後來的我們 (Here| After| Us) - 五月天 (Mayday),27,8.0
2874,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,好好 (想把你寫成一首歌) (Song About You) - 五月天 (Mayday),25,9.0
464,6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=,Don't Let Me Down - The Chainsmokers,23,10.0


# Build a song recommender with personalization

We now create an item similarity based collaborative filtering model that allows us to make personalized recommendations to each user.

# Creating Similarity based Music Recommendation in Python:

In [83]:
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song')

# Use the personalized model to make some song recommendations

In [88]:
#Print the songs for the user in training data
user_id1 = users[1]
#Fill in the code here
user_items1 = is_model.get_user_items(user_id1)
print("------------------------------------------------------------------------------------")
print("Songs played by second user %s:" % user_id2)
print("------------------------------------------------------------------------------------")

for user_item in user_items1:
    print(user_item)

print("----------------------------------------------------------------------")
print("Similar songs recommended for the second user:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id1)

------------------------------------------------------------------------------------
Songs played by second user 6VAZ05qs9gRNM6FZgGvEs9SukGVvBwF/EJ3pMWxgC24=:
------------------------------------------------------------------------------------
眼| 鼻| 口 (Eyes| Nose| Lips) - BIGBANG TAEYANG
Lords of Cardboard - Various Artists
Disco Africa - Soundway
Hip Hop Is Dead(Album Version (Edited)) - Nas
----------------------------------------------------------------------
Similar songs recommended for the second user:
----------------------------------------------------------------------
No. of unique songs for the user: 4
no. of unique songs in the training set: 4722
Non zero values in cooccurence_matrix :99


,user_id,song,score,rank
0,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,IF YOU - BIGBANG,0.055556,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,LOSER - BIGBANG,0.055556,2
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Tilted - Christine And The Queens,0.041667,3
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Church Bells - Carrie Underwood,0.041667,4
4,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,天空之城拉普達出自【天空之城】 - 日本動畫名曲集--波妞的遊園地,0.041667,5
5,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,プラネタリウム - Ai Otsuka (大塚愛),0.041667,6
6,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Area Code - Nick Jonas,0.041667,7
7,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,I Feel You - Wonder Girls,0.041667,8
8,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Who Do You Think Of? - M.O,0.041667,9
9,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Only For Her - Various Artists,0.041667,10


In [90]:
user_id2 = users[7]
#Fill in the code here
user_items2 = is_model.get_user_items(user_id2)
print("------------------------------------------------------------------------------------")
print("Songs played by second user %s:" % user_id2)
print("------------------------------------------------------------------------------------")

for user_item in user_items2:
    print(user_item)

print("----------------------------------------------------------------------")
print("Similar songs recommended for the second user:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id2)

------------------------------------------------------------------------------------
Songs played by second user Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=:
------------------------------------------------------------------------------------
親愛陌生人【土豆網偶像劇[歡迎愛光臨]片頭曲】 - 丁噹 (Della)
道聽塗說 (Remembering you) - 林芯儀 (Shennio Lin)
愛磁場 - 張韶涵 (Angela Chang)
宇宙小姐 - S.H.E
愛旅行的人 - 張韶涵 (Angela Chang)
敢愛敢當 - 丁噹 (Della)
九號球 - [逆轉勝] 五月天∕怪獸 原聲原創紀 ([Second Chance] Soundtrack & Autobiography of Mayday Monster)
刺情 - 張韶涵 (Angela Chang)
Baby - Justin Bieber
Never Forget You - 張韶涵 (Angela Chang)
安靜了 - S.H.E
----------------------------------------------------------------------
Similar songs recommended for the second user:
----------------------------------------------------------------------
No. of unique songs for the user: 11
no. of unique songs in the training set: 4722
Non zero values in cooccurence_matrix :238


,user_id,song,score,rank
0,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,I Wanna Be - Katy B,0.045455,1
1,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,等著你回來 - 田馥甄 (Hebe),0.045455,2
2,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,玩愛之徒 (Love Player) - 蔡依林 (Jolin Tsai),0.045455,3
3,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,我還在原地哭 - 朱俐靜 (Miu Chu),0.045455,4
4,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,SHERO - S.H.E,0.045455,5
5,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,遠路《晟俊與惠珍真愛之歌》 - Various Artists,0.045455,6
6,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,The Spark - Afrojack,0.045455,7
7,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,無路可退 - A-Lin,0.041667,8
8,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,920 - A-Lin,0.032828,9
9,Vgeu+u3vXE0FhQtG/Vr3I/U3V0TX/jzQAEBhi3S3qi0=,One Less Lonely Girl - Justin Bieber,0.032828,10


The lists of both the users in popularity based recommendation is the same but different in case of similarity-based recommendation.

# We can also apply the model to find similar songs to any song in the dataset

In [91]:
is_model.get_similar_items(['U Smile - Justin Bieber'])

no. of unique songs in the training set: 4722
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,We'll Keep Running Forever - Joshua Radin,0.0,1
1,,舒伯特-小夜曲 - 寶寶睡眠音樂,0.0,2
2,,假男友 (The Fake Lovers) - 蔡旻佑 (Evan Yo),0.0,3
3,,不完整的旋律 (Bu Wan Zheng De Xuan Lu) - 王力宏 (Leehom...,0.0,4
4,,Complicity Itself - 証聲音樂圖書館 ECHO MUSIC,0.0,5
5,,The Show - Lenka,0.0,6
6,,As the World Goes Away - Lights & Motion,0.0,7
7,,Touch My Body(零距離接觸) - Mariah Carey,0.0,8
8,,半島鐵盒 - 周杰倫 (Jay Chou),0.0,9
9,,ONLY ONE - 率智哈妮 (EXID),0.0,10
